### Exploratory Data Analysis

In [1]:
import plotly.express as px
import polars as pl

from stocksense.config import config
from stocksense.pipeline import clean, engineer_features

features = config.model.features
date_col = config.model.date_col
aux_cols = ["tic", "datadate", "rdq"]
targets = config.model.targets

In [8]:
data = engineer_features()
data = clean(data)
data.head()

2025-01-10 11:28:13.841 | INFO     | stocksense.pipeline.preprocess:engineer_features:20 - START processing stock data
2025-01-10 11:28:13.844 | SUCCESS  | stocksense.database.schema:create_tables:121 - Tables created successfully
2025-01-10 11:28:16.801 | INFO     | stocksense.pipeline.preprocess:engineer_features:33 - START feature engineering
2025-01-10 11:28:19.488 | INFO     | stocksense.pipeline.preprocess:compute_performance_targets:1178 - Target hit rates: shape: (1, 3)
┌─────────────────┬───────────────┬──────────────┐
│ aggressive_rate ┆ balanced_rate ┆ relaxed_rate │
│ ---             ┆ ---           ┆ ---          │
│ f64             ┆ f64           ┆ f64          │
╞═════════════════╪═══════════════╪══════════════╡
│ 0.369895        ┆ 0.454809      ┆ 0.36882      │
└─────────────────┴───────────────┴──────────────┘
2025-01-10 11:28:19.489 | INFO     | stocksense.pipeline.preprocess:filter_sp500_membership:1120 - START filtering likely SP500 members
2025-01-10 11:28:19.496 

tdq,tic,datadate,rdq,saleq,cogsq,xsgaq,niq,ebitdaq,cshoq,actq,atq,cheq,rectq,invtq,ppentq,lctq,dlttq,ltq,req,seqq,oancfq,ivncfq,fincfq,dvq,capxq,icaptq,surprise_pct,stock_split,n_purch,val_purch,n_sales,val_sales,insider_balance,roa,roi,roe,…,balanced_hit,relaxed_hit,mkt_cap_rank,sector_communication_services,sector_consumer_discretionary,sector_consumer_staples,sector_energy,sector_financials,sector_health_care,sector_industrials,sector_information_technology,sector_materials,sector_real_estate,sector_utilities,pe_sec,momentum_sec_yoy,momentum_sec_qoq,size_sec,size_factor,mkt_rel_sec,roa_sec,pe_mkt_rank,pe_sec_rank,ev_ebitda_mkt_rank,ev_ebitda_sec_rank,saleq_yoy_mkt_rank,saleq_yoy_sec_rank,roa_mkt_rank,roa_sec_rank,fcf_mkt_rank,fcf_sec_rank,der_mkt_rank,der_sec_rank,price_mom_mkt_rank,price_mom_sec_rank,price_yoy_mkt_rank,price_yoy_sec_rank
date,str,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,u32,f64,u32,f64,f64,f64,f64,f64,…,i8,i8,u32,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,f64,f64,f64,f64,i8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2007-03-01,"""A""",2007-01-31,2007-02-15,1280.0,540.0,564.0,150.0,176.0,405.0,3749.0,7193.0,2090.0,671.0,648.0,775.0,1372.0,1500.0,3536.0,1705.0,3657.0,93.0,-93.0,-169.0,0.0,37.0,5157.0,0.011,0,9,0.616,3,15.993,15.377,8.911442,2.908668,0.17528,…,0,0,304,0,0,0,0,0,1,0,0,0,0,0,0.500932,8.521066,2.638213,1.3791e6,0,0.660437,3.577044,3.0,3.0,7.0,5.0,6.0,5.0,7.0,6.0,4.0,5.0,4.0,6.0,4.0,5.0,1.0,2.0
2007-06-01,"""A""",2007-04-30,2007-05-14,1320.0,540.0,586.0,123.0,194.0,395.958,3791.0,7283.0,2050.0,718.0,650.0,777.0,1554.0,1500.0,3728.0,1881.0,3555.0,302.0,-36.0,-320.0,0.0,42.0,5055.0,-0.023,0,9,0.616,4,11.293,10.677,8.911163,2.433234,0.18256,…,0,0,281,0,0,0,0,0,1,0,0,0,0,0,0.5889,24.387775,11.05679,1.4918e6,1,0.726781,4.746319,3.0,3.0,7.0,5.0,4.0,2.0,7.0,6.0,8.0,7.0,4.0,6.0,7.0,8.0,3.0,3.0
2007-09-01,"""A""",2007-07-31,2007-08-14,1374.0,563.0,580.0,185.0,231.0,386.548,3281.0,7024.0,1486.0,738.0,674.0,787.0,1510.0,1500.0,3736.0,2069.0,3288.0,176.0,-258.0,-484.0,0.0,36.0,4788.0,-0.01,0,8,0.133,6,17.04,16.907,8.6418,3.863826,0.184611,…,0,0,292,0,0,0,0,0,1,0,0,0,0,0,0.652592,17.5465,0.882991,1.4038e6,0,0.716937,3.892267,4.0,4.0,7.0,5.0,6.0,5.0,7.0,6.0,6.0,6.0,4.0,6.0,2.0,1.0,5.0,5.0
2007-12-01,"""A""",2007-10-31,2007-11-15,1446.0,611.0,598.0,180.0,237.0,370.0,3671.0,7554.0,1826.0,735.0,643.0,801.0,1663.0,2087.0,4320.0,2580.0,3234.0,398.0,-69.0,-7.0,0.0,39.0,5321.0,0.022,0,0,0.0,8,19.79,19.79,8.445856,3.382823,0.197279,…,0,0,285,0,0,0,0,0,1,0,0,0,0,0,0.626737,20.618915,5.97764,1.4828e6,0,0.67524,3.634633,4.0,3.0,7.0,6.0,5.0,3.0,7.0,5.0,8.0,8.0,5.0,7.0,9.0,7.0,7.0,6.0
2008-03-01,"""A""",2008-01-31,2008-02-13,1393.0,580.0,617.0,120.0,196.0,368.0,5070.0,7459.0,3148.0,726.0,674.0,801.0,2674.0,626.0,4286.0,2657.0,3173.0,4.0,-295.0,-168.0,0.0,34.0,3799.0,0.021,0,0,0.0,9,14.61,14.61,8.151227,3.158726,0.191617,…,0,0,311,0,0,0,0,0,1,0,0,0,0,0,0.903003,7.695396,-7.595645,1.3298e6,0,0.605916,3.103504,4.0,3.0,7.0,5.0,5.0,3.0,7.0,6.0,2.0,2.0,5.0,7.0,2.0,2.0,6.0,5.0


In [13]:
df = data.with_columns(
    pl.col("fwd_return_4Q").clip(-200, 200).alias("fwd_return_4Q"),
    pl.col("risk_return_4Q").clip(-100, 100).alias("risk_return_4Q"),
)
fig = px.histogram(
    df, x=["fwd_return_4Q", "risk_return_4Q"], title="Distribution", nbins=100, barmode="overlay"
)
fig.show()

In [4]:
mcd = data.filter(pl.col("tic") == "YUM")
mcd

tdq,tic,datadate,rdq,saleq,cogsq,xsgaq,niq,ebitdaq,cshoq,actq,atq,cheq,rectq,invtq,ppentq,lctq,dlttq,ltq,req,seqq,oancfq,ivncfq,fincfq,dvq,capxq,icaptq,surprise_pct,stock_split,n_purch,val_purch,n_sales,val_sales,insider_balance,roa,roi,roe,…,ltda_2y,pe_yoy,pe_2y,pb_yoy,pb_2y,ps_yoy,ps_2y,eps_yoy,eps_2y,ev_ebitda_yoy,ltcr_yoy,itr_yoy,rtr_yoy,atr_yoy,size_yoy,roa_sec_qoq,roa_sec_yoy,f_score,forward_vol_yoy,forward_vol_sos,forward_vol_qoq,excess_return_1Q,sharpe_ratio_1Q,risk_return_1Q,excess_return_2Q,sharpe_ratio_2Q,risk_return_2Q,excess_return_3Q,sharpe_ratio_3Q,risk_return_3Q,excess_return_4Q,sharpe_ratio_4Q,risk_return_4Q,fwd_return,excess_return,sharpe_ratio,risk_return
date,str,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,u32,f64,u32,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2007-03-01,"""YUM""",2006-12-31,2007-02-12,3016.0,2120.0,443.0,232.0,453.0,530.0,901.0,6353.0,325.0,220.0,93.0,3631.0,1724.0,2045.0,4916.0,1437.0,1437.0,295.0,-176.0,-76.0,40.0,291.0,3482.0,0.054,0,0,0.0,7,10.802,10.802,12.97025,6.662837,0.573417,…,NaN,3.071736,null,12.388907,NaN,8.986742,null,13.441292,null,1.669279,103.839795,-4.383214,8.732754,-1.754273,1.258251,-4.739447,8.712918,5,1.81229,1.489239,1.084487,5.600592,5.16428,10.530256,9.266337,8.544446,14.75092,19.383489,13.015698,18.005365,29.645544,16.358062,16.582675,26.814296,19.383489,13.015698,18.005365
2007-06-01,"""YUM""",2007-03-31,2007-05-01,2223.0,1542.0,274.0,194.0,407.0,524.0,1041.0,6379.0,341.0,278.0,97.0,3570.0,1488.0,2208.0,4920.0,1459.0,1459.0,341.0,-42.0,-277.0,40.0,93.0,3667.0,0.099,0,0,0.0,14,5.478,5.478,13.29362,5.290428,0.58122,…,17.499715,18.21296,null,17.305894,39.707797,24.451279,null,13.427863,null,13.119532,-14.637681,-5.165468,5.76233,-4.40995,1.266497,6.106867,9.509898,4,1.914891,1.702851,1.799144,-0.985628,-0.547832,-2.307626,8.344666,4.638132,3.28782,16.45971,9.665973,4.820542,25.209564,13.165014,7.329741,8.208665,16.45971,9.665973,4.820542
2007-09-01,"""YUM""",2007-06-30,2007-07-11,2367.0,1642.0,306.0,214.0,419.0,521.0,1104.0,6430.0,436.0,252.0,98.0,3552.0,1695.0,2088.0,4973.0,1457.0,1457.0,252.0,-74.0,-89.0,79.0,124.0,3545.0,0.076,1,0,0.0,17,9.413,9.413,13.530327,6.036671,0.597117,…,17.978904,15.687107,null,22.390525,34.768278,20.820814,null,14.553204,null,13.686995,-15.469023,-6.541634,-1.550681,-1.92138,1.061968,18.096718,36.440277,4,2.047201,2.082258,1.657973,13.330495,8.040238,9.249603,16.759604,10.108492,7.275507,26.707222,12.826085,9.207945,25.274567,12.345912,6.564224,19.17332,26.707222,12.826085,9.207945
2007-12-01,"""YUM""",2007-09-30,2007-10-08,2564.0,1761.0,292.0,270.0,511.0,512.0,1165.0,6598.0,507.0,250.0,96.0,3600.0,1866.0,2124.0,5176.0,1422.0,1422.0,576.0,-140.0,-368.0,77.0,174.0,3546.0,0.119,0,0,0.0,20,13.894,13.894,13.792058,7.614213,0.639944,…,18.926604,5.857372,null,9.97847,53.229461,9.356404,-13.630451,15.157969,null,7.533914,17.199044,-4.85052,-0.001049,1.643236,1.285804,10.143046,31.47752,4,2.821542,2.110907,2.491781,3.722422,1.49388,-1.59581,12.303426,4.937603,1.77462,11.669343,5.528117,-0.514096,13.967799,4.950413,-5.492781,-1.085209,11.669343,5.528117,-0.514096
2008-03-01,"""YUM""",2007-12-31,2008-02-04,3262.0,2286.0,496.0,231.0,480.0,499.0,1481.0,7242.0,789.0,225.0,128.0,3849.0,2062.0,2924.0,6103.0,1139.0,1139.0,398.0,-176.0,56.0,77.0,351.0,4063.0,0.05,0,1,0.397,10,52.2,51.803,12.551781,5.685454,0.798068,…,39.514914,2.930248,6.091993,43.255972,61.003821,4.227432,13.594083,17.168804,32.917804,4.647387,300.0,-16.468418,0.870166,-4.237736,1.495661,-1.909498,35.383382,5,2.89054,2.019357,1.730596,10.996881,6.354388,8.75172,9.166835,5.296922,3.864821,12.628194,6.253573,-3.902243,21.162932,7.321446,-5.238059,-7.880021,12.628194,6.253573,-3.902243
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

In [8]:
data.filter(pl.col("tic") == "NVDA").select(
    ["tdq", "fwd_return_3Q", "risk_return_3Q", "fwd_return_4Q", "risk_return_4Q"]
).tail(11)

tdq,fwd_return_3Q,risk_return_3Q,fwd_return_4Q,risk_return_4Q
date,f64,f64,f64,f64
2022-06-01,-0.950433,-0.252648,52.145192,13.9644
2022-09-01,96.810305,25.319246,200.0,61.334766
2022-12-01,153.846034,41.739487,164.975952,53.725829
2023-03-01,98.007867,31.764888,155.889126,55.370632
2023-06-01,44.232892,18.704916,126.35727,46.005311
…,…,…,…,…
2023-12-01,158.87167,52.031133,178.63735,54.725834
2024-03-01,57.718003,15.844878,70.369899,null
2024-06-01,26.123682,7.553415,null,null


In [5]:
# Get null counts for each column and sort
null_counts = (
    data.select(features)
    .to_pandas()
    .isnull()
    .sum()
    .reset_index()
    .rename(columns={"index": "column", 0: "null_count"})
    .sort_values("null_count", ascending=False)
    .assign(null_pct=lambda x: (x["null_count"] / len(data) * 100).round(2))
)
print("\nNull value counts and percentages:")
print(null_counts)


Null value counts and percentages:
                column  null_count  null_pct
58            ltcr_yoy        2152      5.86
56            ltda_yoy        1703      4.64
44           dlttq_yoy        1694      4.62
57       ev_ebitda_yoy        1462      3.98
36           ev_ebitda        1420      3.87
..                 ...         ...       ...
25           high_fear           0      0.00
26            low_fear           0      0.00
1              n_sales           0      0.00
41              niq_2y           0      0.00
75  sector_real_estate           0      0.00

[76 rows x 3 columns]


In [6]:
# Get null counts per company
companies_with_nulls = (
    data.select(["tic"] + features)
    .to_pandas()
    .groupby("tic")
    .apply(lambda x: x.isnull().sum().sum())
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={0: "total_nulls"})
    .assign(
        null_pct=lambda x: (
            x["total_nulls"] / (len(data.columns) * x.groupby("tic").size().values) * 100
        ).round(2)
    )
)

print(f"\nTotal companies with missing data: {(companies_with_nulls['total_nulls'] > 0).sum()}")
print("\nTop 10 companies with most missing data:")
print(companies_with_nulls.head(10))


Total companies with missing data: 249

Top 10 companies with most missing data:
    tic  total_nulls  null_pct
0  ALGN          205    106.77
1  ISRG          204    106.25
2  EXPD          204    106.25
3  TROW          204    106.25
4  GRMN          195    101.56
5  MNST          191     99.48
6  FFIV          172     89.58
7  LIFE          165     85.94
8  MPWR          156     81.25
9   CMG          156     81.25


/var/folders/b8/1sybx4y52l7fb3b8glvm3jk40000gn/T/ipykernel_35137/875865165.py:5: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [7]:
data.filter(pl.col("tic") == "L")

tdq,tic,datadate,rdq,saleq,cogsq,xsgaq,niq,ebitdaq,cshoq,actq,atq,cheq,rectq,invtq,ppentq,lctq,dlttq,ltq,req,seqq,oancfq,ivncfq,fincfq,dvq,capxq,icaptq,surprise_pct,stock_split,n_purch,val_purch,n_sales,val_sales,insider_balance,roa,roi,roe,…,ps_yoy,ps_2y,eps_yoy,eps_2y,ev_ebitda_yoy,ev_ebitda_2y,ltcr_yoy,itr_yoy,rtr_yoy,atr_yoy,size_yoy,f_score,f_score_gr1,f_score_gr4,forward_vol_yoy,forward_vol_sos,forward_vol_qoq,fwd_return,risk_return,excess_return,performance_positive,risk_return_positive,fwd_sharpe_ratio,risk_return_2Q,risk_return_3Q,risk_return_4Q,sector_communication_services,sector_consumer_discretionary,sector_consumer_staples,sector_energy,sector_financials,sector_health_care,sector_industrials,sector_information_technology,sector_materials,sector_real_estate,sector_utilities
date,str,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,u32,f64,u32,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,i8,i8,f64,f64,f64,f64,f64,f64,i8,i8,f64,f64,f64,f64,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
2008-03-01,"""L""",2007-12-31,2008-02-11,3592.0,2857.0,0.0,384.0,0.0,529.684,0.0,76079.0,12872.0,11677.0,224.0,10425.0,0.0,6900.0,54590.0,13626.0,17591.0,1296.8,-1273.1,-2.8,82.5,901.7,28389.0,-0.239,0,1,0.191,4,1.724,1.533,0.02571,0.013526,0.111193,…,12.902874,34.885943,-19.334673,69.929798,NaN,NaN,200.0,-13.793438,-15.299251,-24.908028,-0.093202,5,0,-1,4.614202,1.761551,1.867523,-0.397332,-8.611077,-0.034295,0,0,-0.743249,4.959346,-9.255767,-8.611077,0,0,0,0,1,0,0,0,0,0,0
2008-06-01,"""L""",2008-03-31,2008-04-28,3612.0,2661.0,0.0,555.0,0.0,529.702,0.0,76892.0,11667.0,11959.0,0.0,11086.0,0.0,7093.0,55847.0,13286.0,17257.0,1487.0,-1230.0,-200.0,82.0,846.0,28138.0,-0.158,0,0,0.0,1,0.082,0.082,0.024203,0.019724,0.10784,…,13.014081,51.193873,-27.442456,39.36741,NaN,NaN,200.0,11.309978,8.702655,-1.10861,0.017638,5,0,1,4.976012,5.668694,1.660324,-0.522072,-10.491769,-0.113731,0,0,-2.285586,-18.65718,-8.45061,-10.491769,0,0,0,0,1,0,0,0,0,0,0
2008-09-01,"""L""",2008-06-30,2008-07-28,3922.0,2727.0,0.0,4859.0,0.0,436.268,0.0,74782.0,10375.0,11965.0,0.0,11782.0,0.0,7137.0,53747.0,13497.0,16781.0,1344.0,-1210.0,-173.0,83.0,933.0,28172.0,-0.16,0,1,0.492,4,2.734,2.242,0.083001,0.172476,0.369883,…,-16.715292,-3.152492,199.489123,300.0,NaN,NaN,-85.054034,NaN,28.610126,14.89984,-0.546362,7,2,3,5.018585,6.277926,7.985833,-0.327632,-6.528371,-0.070222,0,0,-1.399238,-5.264309,-7.203086,-6.528371,0,0,0,0,1,0,0,0,0,0,0
2008-12-01,"""L""",2008-09-30,2008-10-27,2970.0,2929.0,0.0,-137.0,0.0,436.089,0.0,73417.0,9506.0,12201.0,0.0,12809.0,0.0,7401.0,53508.0,12461.0,15741.0,-1454.0,1494.0,24.0,28.0,1158.0,27310.0,-1.5,0,0,0.0,0,0.0,0.0,0.077107,-0.005016,0.359634,…,-55.743324,-46.738337,196.519991,285.704755,NaN,NaN,-200.0,NaN,-4.343053,-9.886907,-0.646001,4,-3,-1,3.214735,4.384858,4.661004,0.488965,15.21013,0.185788,1,1,5.779258,0.503758,5.307936,15.21013,0,0,0,0,1,0,0,0,0,0,0
2009-03-01,"""L""",2008-12-31,2009-02-09,2743.0,3516.0,0.0,-958.0,0.0,435.092,0.0,69857.0,8867.0,11672.0,0.0,12876.0,0.0,8187.0,52735.0,9839.0,13126.0,1994.0,-2268.0,176.0,26.0,1060.0,25309.0,-5.0,0,0,0.0,0,0.0,0.0,0.061826,-0.037852,0.329042,…,-57.926143,-52.497407,168.812902,116.838807,NaN,NaN,99.234318,200.0,-21.355777,-17.362575,-0.759124,5,1,0,2.519552,3.215018,4.121507,0.837111,33.224616,0.330937,1,1,13.134767,11.145129,23.336302,33.224616,0,0,0,0,1,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022-12-01,"""L""",2022-09-30,2022-11-29,3461.0,1665.0,383.0,130.0,282.0,240.76,10344.0,75111.0,886.0,9458.0,0.0,9949.0,42522.0,8475.0,60458.0,15377.0,14653.0,1246.0,-993.0,-303.0,23.0,0.0,21025.392,0.0,0,0,0.0,6,0.173,0.173,0.012981,0.006183,0.066539,…,7.43907,7.662498,-36.153086,201.876143,52.743225,170.610572,15.812824,NaN,-1.670822,10.193546,-0.746214,8,2,2,1.166898,1.369194,1.149776,0.117865,10.100698,0.

In [7]:
data.filter(pl.col("risk_return_4Q") > 70).select(
    ["tic", "tdq", "fwd_return_4Q", "forward_vol_yoy", "risk_return_4Q"]
)

tic,tdq,fwd_return_4Q,forward_vol_yoy,risk_return_4Q
str,date,f64,f64,f64
"""AAL""",2013-09-01,154.136085,2.126597,72.48016
"""ABMD""",2017-09-01,165.335916,2.103878,78.586269
"""AMD""",2016-03-01,200.0,4.775459,86.891669
"""BBWI""",2020-06-01,200.0,3.755184,74.477339
"""BKNG""",2010-06-01,168.915432,2.390081,70.673526
…,…,…,…,…
"""TSLA""",2019-09-01,200.0,5.147724,97.705849
"""TSLA""",2019-12-01,200.0,5.576061,100.0
"""TSLA""",2020-03-01,200.0,5.316739,82.419614


In [8]:
data.filter(pl.col("risk_return") > 40).select(
    ["tic", "tdq", "avg_fwd_return", "forward_vol", "risk_return"]
).sort("tdq").tail(20)

tic,tdq,avg_fwd_return,forward_vol,risk_return
str,date,f64,f64,f64
"""NVDA""",2023-09-01,1.473149,0.031268,47.113693
"""PGR""",2023-09-01,0.625629,0.013195,47.413705
"""BK""",2023-12-01,0.556011,0.010986,50.610972
"""BSX""",2023-12-01,0.520267,0.010517,49.467115
"""COST""",2023-12-01,0.557523,0.012445,44.798358
…,…,…,…,…
"""TMUS""",2023-12-01,0.440126,0.01,44.012593
"""TRGP""",2023-12-01,0.883907,0.014261,61.980651
"""TT""",2023-12-01,0.712079,0.014324,49.71244
